<a href="https://colab.research.google.com/github/zjuer-dm/colab/blob/main/2d_gaussian_splatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before proceeding, please ensure that your notebook is connected to a GPU runtime.

In [ ]:
!apt-get update
!apt-get install -y build-essential libglm-dev

In [ ]:
# 1. 克隆主仓库及其所有子模块
!echo ">>> 1. Cloning repository..."
!git clone --recursive https://github.com/hbb1/2d-gaussian-splatting.git
%cd 2d-gaussian-splatting

# 2. 安装第一个缺失的系统依赖 (用于解决第一个错误)
!echo ">>> 2. Installing libglm-dev..."
!apt-get update && apt-get install -y libglm-dev

# 3. 【关键修复】为 simple-knn 源码打补丁 (用于解决 FLT_MAX 错误)
!echo ">>> 3. Patching simple-knn source code..."
!sed -i '1i#include <cfloat>' /content/2d-gaussian-splatting/submodules/simple-knn/simple_knn.cu

# 4. 现在，执行完整的 pip 安装
!echo ">>> 4. Running pip install..."
!pip install ./submodules/diff-surfel-rasterization
!pip install ./submodules/simple-knn


!echo ">>> Installation finished."
!pip install -q plyfile

In [ ]:
%cd /content/2d-gaussian-splatting/

In [ ]:
import os
# os.environ['CUDA_HOME'] = '/usr/local/cuda' #ensure that the CUDA path is correct
# !pip install ninja
# !pip install submodules/simple-knn --verbose
# !pip install submodules/diff-surfel-rasterization --verbose

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 安装 COLMAP 及其所有依赖项。这是处理数据的关键步骤。
# 更新包列表
!sudo apt-get update
# 安装 COLMAP
!sudo apt-get install -y colmap

In [ ]:
# 安装 Xvfb (X Virtual Framebuffer)，一个虚拟屏幕工具
!apt-get update
!apt-get install -y xvfb

In [ ]:
import os
from google.colab import drive



# --- 1. 设置您的项目路径 ---
# (请确保这里的路径是正确的)
project_path = "/content/drive/MyDrive/try"

# 定义 COLMAP 处理所需的各个路径
db_path = os.path.join(project_path, "colmap.db")
image_path = os.path.join(project_path, "images")
sparse_path = os.path.join(project_path, "sparse")

# 创建稀疏重建的输出目录
os.makedirs(sparse_path, exist_ok=True)

# --- 2. 执行 COLMAP 命令 (使用 xvfb-run 避免图形界面错误) ---

# a. 特征提取
print(">>> Running COLMAP: Feature Extraction...")
!xvfb-run -a colmap feature_extractor \
    --database_path "{db_path}" \
    --image_path "{image_path}"

# b. 特征匹配
print(">>> Running COLMAP: Exhaustive Matching...")
!xvfb-run -a colmap exhaustive_matcher \
    --database_path "{db_path}"

# c. 场景重建 (Mapping) - 此步骤通常不需要虚拟屏幕
print(">>> Running COLMAP: Mapping...")
!colmap mapper \
    --database_path "{db_path}" \
    --image_path "{image_path}" \
    --output_path "{sparse_path}"

print(f"🎉 COLMAP processing complete! Your dataset is ready in: {project_path}")
print("You can now proceed to the training step.")

In [ ]:
!python /content/2d-gaussian-splatting/train.py -s "{colmap_output_path}"

In [ ]:
%cd /content/2d-gaussian-splatting/

For Colab to access your COLMAP processed data, please copy the COLMAP output to your Google Drive and enter the full COLMAP source path (e.g. /content/drive/MyDrive/COLMAP_out)

In [ ]:
COLMAP_PATH = str(input('Please enter COLMAP output path:'))

In [ ]:
!python train.py -s $COLMAP_PATH

Training is complete, to use render.py, please enter the desired model_path located in your Drive: /content/drive/MyDrive/2d_gaussian_splatting/output/{10 digit UUID}: (e.g: /content/drive/MyDrive/2d_gaussian_splatting/output/8d812792-1)

In [ ]:
MODEL_PATH = str(input('Please enter the model path:'))

In [ ]:
!pip3 install open3d==0.18.0 mediapy==1.1.2 lpips==0.1.4 scikit-image==0.21.0 tqdm==4.66.2 trimesh==4.3.2 opencv-python

In [ ]:
!python render.py -m $MODEL_PATH -s $COLMAP_PATH

Training and rendering is complete, please navigate to your drive and see the model outputs under the directory "2d_gaussian_splatting/".